In [3]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

Xtrain = np.array(pd.read_csv('csv/x_train_significant.csv', sep=','))
ytrain = np.array(pd.read_csv('csv/y_train.csv', sep=',', header=None))
Xtest = np.array(pd.read_csv('csv/x_test_significant.csv', sep=','))
ytest = np.array(pd.read_csv('csv/y_test.csv', sep=',', header=None))

In [ ]:
sample_leaf_options = [1,5,10,50,100,200,500]
max_score = -1
max_score_leaf = 0
fewest_bad_pairs = ytest.shape[0]/2
fewest_bad_pairs_leaf = 0
for min_leaf in sample_leaf_options:
    clf = RandomForestClassifier(n_estimators=200, max_features="sqrt", min_samples_leaf=min_leaf, oob_score = True)
    clf = clf.fit(Xtrain, np.ravel(ytrain))

    p = clf.predict(Xtest)
    score = clf.score(Xtest, ytest, sample_weight=None)
    if (score > max_score):
        max_score = score
        max_score_leaf = min_leaf

    bad_pair = 0
    for i in range(int(p.shape[0]/2)):
        if p[2*i]==p[2*i+1]:
            bad_pair+=1
    if (bad_pair < fewest_bad_pairs):
        fewest_bad_pairs = bad_pair
        fewest_bad_pairs_leaf = min_leaf
print('Best rate:', max_score, max_score_leaf, 'min_leaf')
print('Fewest bad pairs:', fewest_bad_pairs, fewest_bad_pairs_leaf, 'min_leaf')

In [ ]:
n_estimators = [50,100,200,500,750,1000]
max_score = -1
max_score_trees = 0
fewest_bad_pairs = p.shape[0]/2
fewest_bad_pairs_trees = 0
for n_trees in n_estimators:
    clf = RandomForestClassifier(n_estimators=n_trees, max_features="sqrt", 
                                 min_samples_leaf=max_score_leaf, oob_score = True)
    clf = clf.fit(Xtrain, np.ravel(ytrain))

    p = clf.predict(Xtest)
    score = clf.score(Xtest, ytest, sample_weight=None)
    if (score > max_score):
        max_score = score
        max_score_trees = min_leaf

    bad_pair = 0
    for i in range(int(p.shape[0]/2)):
        if p[2*i]==p[2*i+1]:
            bad_pair+=1
    if (bad_pair < fewest_bad_pairs):
        fewest_bad_pairs = bad_pair
        fewest_bad_pairs_trees = min_leaf
print('Best rate:', max_score, max_score_trees, 'trees')
print('Fewest bad pairs:', fewest_bad_pairs, fewest_bad_pairs_trees, 'trees')

In [4]:
clf = RandomForestClassifier(n_estimators=500, max_features="sqrt", 
                                 min_samples_leaf=100, oob_score = True)
clf = clf.fit(Xtrain, np.ravel(ytrain))
p = clf.predict(Xtest)
probs = clf.predict_proba(Xtest)

In [5]:
games_test = pd.read_csv('games_test.csv', header=None)
wins = {}
for i in range(len(games_test)):
    away_win = (probs[2*i][1]+probs[2*i+1][0])/2
    home_win = (probs[2*i][0]+probs[2*i+1][1])/2
    away = games_test.loc[i,0]
    home = games_test.loc[i,2]
    if away not in wins:
        wins[away] = away_win
    else:
        wins[away] += away_win
    if home not in wins:
        wins[home] = home_win
    else:
        wins[home] += home_win
[(team, wins[team]) for team in sorted(wins, key=wins.get)]

[('Los Angeles Lakers', 29.367186892164895),
 ('Philadelphia 76ers', 29.905115416817761),
 ('Denver Nuggets', 30.260341266133921),
 ('Minnesota Timberwolves', 30.671231871699366),
 ('New York Knicks', 30.798198541428096),
 ('Charlotte Hornets', 31.501948630750917),
 ('Detroit Pistons', 31.643766864816488),
 ('Orlando Magic', 31.832153583949552),
 ('Sacramento Kings', 31.919075665431084),
 ('Miami Heat', 36.612507031921069),
 ('Phoenix Suns', 37.805993381871296),
 ('Milwaukee Bucks', 38.838380507351857),
 ('Indiana Pacers', 40.902488406476486),
 ('Utah Jazz', 41.784329125719431),
 ('New Orleans Pelicans', 43.465039905069915),
 ('Brooklyn Nets', 43.465987785578015),
 ('Oklahoma City Thunder', 43.565331706614842),
 ('Boston Celtics', 44.107909268477229),
 ('Washington Wizards', 44.961540243660281),
 ('Memphis Grizzlies', 45.301906292612159),
 ('Portland Trail Blazers', 45.839513511764103),
 ('Chicago Bulls', 46.617064905474592),
 ('Toronto Raptors', 46.839125945165691),
 ('Houston Rockets

In [6]:
import pred_to_record
pred_to_record.pred_to_record(p)
pred_to_record.pred_to_record([prob[1] for prob in probs])
pred_to_record.rms(p)
pred_to_record.rms_round([prob[1] for prob in probs])

10.289152864384253